**Fin 585R**  
**Diether**  
**Problem Set**  
**Time Series Tests of the CAPM**  

**Overview**  

In this problem set you test the CAPM using the time series framework. Specifically, you test whether the CAPM holds with respect to *momentum portfolios* (Jegadeesh and Titman, 1993, "Returns to Buying Winners and Selling Losers: Implications for Stock Market Efficiency") using the **time series testing framework.** Momentum portfolios are formed based on past returns. Specifically, momentum portfolios are most commonly formed based on the cumulative return from months $t-12$ to $t-2$:

$$
r_{i,t-12:t-2} \approx \sum_{x=2}^{12} \log(1+r_{i,t-x})
$$

I formed equal-weight momentum portfolios for you; the portfolios are available for download on *Learning Suite* or you can download it directly from the following link: [momentum portfolios](https://diether.org/prephd/10-port_mom_ew.csv)). Note, that in my data the returns for these portfolios are in percent; I have multiplied the raw returns by 100. Also, to test the CAPM you are going to need a proxy for the market portfolio and for the riskfree rate. Data from these can be found at [Ken French's Data Library](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). For your convenience I have created a csv file that contains both these variables, and it can be loaded directly into a dataframe from my website (see the code below). The `dataframe` contains the excess return on a proxy for the market portfolio (`exmkt`), a proxy for the riskfree rate (`rf`), and some other portfolios you can ignore. The returns from Ken French's library are also in percent: raw returns multiplied by 100.

For questions that require some write-up, create a markdown cell (use the Cell Toolbar)  and write your answer in the markdown cell (this cell is a markdown cell and here is a [markdown cheat sheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)). 

**Tasks and Questions**  

1. Compute and report the sample mean, standard deviation, and t-test for the mean = 0 for the *excess returns* of the momentum portfolios ($r_p-r_f$). <br><br>

2. Test the CAPM by running a time series CAPM regression for each of the momentum portfolios: <br><br>
$$
r_{pt} - r_{ft} = \alpha_p + \beta_{pM}( r_{Mt} - r_{ft}) + \epsilon_{it}
$$
<br>Consolidate all your regression results into one table (hint, go take a look at the jupyter notebook on estimating regressions in `pandas` and `statsmodels`). <br><br>

3. Interpret the regression results from question 2). What can you infer? Can you reject that
the CAPM holds? Is the market portfolio the tangency portfolio? <br><br>

4. Create a spread portfolio:<br><br>
$$
r_{spread,t} = r_{4t} - r_{0t}.
$$
<br>Test the CAPM using this portfolio. Can you reject the CAPM?<br><br>

5. Estimate the security market line using the data available for this homework. Specifically, estimate the following line:  <br><br>
$$
E(r_p) = r_f + \beta_{p}\bigl[E(r_M) - r_f\bigr]
$$
<br> You don't need to plot the estimated line, but report your estimates of $r_f$ and $E(r_M) - r_f$. <br><br>

6. Why is the intercept in time series CAPM regression called an *average abnormal return*? Briefly explain.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [ ]:
port = pd.read_csv('http://diether.org/prephd/10-port_mom_ew.csv',parse_dates=['caldt'])
port

In [ ]:
fac = pd.read_csv('http://diether.org/prephd/10-factor.csv',parse_dates=['caldt'])
fac.tail()

<br>

**Merge factor data into portfolio data**<br><br>

In [ ]:
port = port.merge(fac,on='caldt',how='inner')
port = port.set_index('caldt')
port.head()

In [ ]:
from finance_byu.summarize import summary
summary(port).loc[['mean','std','tstat']].round(3)

<br>

**Question 1** <br><br> 

In [ ]:
names = ['exp0','exp1','exp2','exp3','exp4']
port[names] = port[[s[2:] for s in names]].sub(port['rf'],axis='index')
port.head()

**Table 1: Summary Statistics for Excess Returns**

In [ ]:
summary(port[names]).loc[['mean','std','tstat']].round(3)

<br>

**Question 2**

$$
r_{pt} - r_{ft} = \alpha_p + \beta_{pM}( r_{Mt} - r_{ft}) + \epsilon_{it}
$$
<br><br>

In [ ]:
reg0 = smf.ols('exp0 ~ 1 + exmkt',data=port).fit()
reg0.summary()

In [ ]:
reg = [smf.ols(f'{y} ~ exmkt',data=port).fit() for y in names]

<br>

**Table: CAPM Regression Results**

<br>

In [ ]:
from finance_byu.regtables import Regtable
Regtable(reg,stat='tstat',sig='coeff').render()

**GRS Test**

+ We can also do a F-test under the null that all the alpha = 0. $\leftarrow$ joint F-test.<br><br>

+ It's technically the correct statistical thing to do.<br><br>

+ Fortunately, it's a function in the BYU Finance library: [GRS Docs](https://fin-library.readthedocs.io/en/latest/statistics.html#statistics)<br><br>

In [ ]:
from finance_byu.statistics import GRS

grsstat,pval,tbl = GRS(port,names,['exmkt'])

print(f'GRS = {grsstat:.2f} and p-value = {pval:.5g}\n')

<br>

**Question 3**

Yes, we can reject the CAPM. Well, really we can jointly reject the CAPM and the efficient market hypothesis. So it could be that the market is just inefficient and the CAPM holds (and we don't know which is the issue, technically). Also, we are using a proxy for the market portfolio. So really we can only reject the hypothesis that our proxy for the market portfolio is the tangency portfolio. Look at the alphas. We have a whole bunch of significant alphas and the CAPM only holds only if all the true alphas are equal to zero. For example, the t-stat of the alpha for the past winner portfolio (`p4`) is greater than 6 and for the past loser portfolio in absolute value. These are big t-stats and the magnitude of the alphas are big too: -0.86% per month for the past loser portfolio (less than -10% per year).<br><br>

<br>

**Question 4**

$$
r_{spread,t} = r_{5,t} - r_{1,t}.
$$
<br><br>

In [ ]:
port['spread'] = port['p4'] - port['p0']

<br>

**Add the Spread Portfolio to the Regression Estimation.**

I could just estimate the spread regression, but all restimate all of them to show how I add the spread portfolio to the regression list comprehension.<br><br>

In [ ]:
reg = [smf.ols(f'{y} ~ exmkt',data=port).fit() for y in names + ['spread']]

**Tabl 3: CAPM Regression Results With A Spread Portfolio** 

In [ ]:
Regtable(reg,stat='tstat',sig='coeff').render()

Yes, we can reject the CAPM (with the same caveats as question 3). The alpha on the spread portfolio is significant: t-stat = 8.8. 

<br>

**Question 5**

+ For estimated SML, just replace the expected returns with their sample analogs:<br><br>
$$
\bar{r}_p = \bar{r}_f + \hat{\beta_i}\Bigl(\bar{r_M} - \bar{r}_f \Bigr)
$$

+ Where $\bar{r}_p$ and $\hat{\beta_i}$ are the dependent and independent variables.<br><br>

+ Just grab the sample averages from your descriptive statistics:<br><br>
$$
\bar{r}_p = 0.27 + \hat{\beta_i}(0.67)
$$


<br>

**Extra Coding: Vizualizing the Test**

Please work through this part of the notebook on your own:

I find often find it useful to visualize tests of the CAPM. Specifically compare the prediction of the CAPM with the actual results graphically. The prediction of the CAPM is the the security market line (SML):

$$
E(r_p) = r_f + \beta_{p}\bigl(E(r_M)-r_f\bigr)
$$

We can compare our average return and estimated beta points with the the SML. Well, really we have to compare our average return-estimated beta points with the estimated SML:

$$
\overline{r}_p = \overline{r}_f + \hat{\beta}_{p}\bigl(\overline{r}_M - \overline{r}_f\bigr)
$$

How do you do this in python? `Python` doesn't have functional graphing capibilities (well, at least not in `pandas` or `matplotlib`. So we need to create a little matrix with the following columns: average returns (not net of the risk free rate), estimated betas, and then create a columns with the estimated SML for all the points. In addition, we probably want to additional aboservations that for the estimate SML: beta = 0 and beta = end of the range your interested (maybe 1.5). You want your estimated SML to extend over a nice looking range.


**Step 1:** Get the estimated betas and save them. How? You can access the estimated coefficient from a statsmodels regressions using the `.params` method. I will use a list comprehension and save them to a list

In [ ]:
[smf.ols(f'{y} ~ exmkt',data=port).fit().params for y in names]

In [ ]:
beta = [smf.ols(f'{y} ~ exmkt',data=port).fit().params[1] for y in names]
beta

**Step 2:** Create a dataframe with the estimated betas and the raw average returns.

In [ ]:
raw = port[['p0','p1','p2','p3','p4']]
raw.head()

Now I can create a dataframe with the raw average returns. Note, I use a little trick here. the output of the `.mean()` method is a `Series`. I change it to a DataFrame using the `to_frame` method:

In [ ]:
est = raw.mean().to_frame()
est = est.rename(columns={0:'r_avg'})
est

Next, add the estimated betas as a column. Those two columns give use the scatter-plot of average return and estimated beta.

In [ ]:
est['beta'] = beta
est

**Step 3:** Create a column with an estimated SML.

(a) I need the average excess return on the market and the average riskfree rate. (b) I then can just make a new column based on the sml formula:

```
avg_rf + est['beta']*avg_exmkt
```

(c) At the same time I am going to add some new rows to the dataframe (`est`) where beta = 0 and beta = 1.5. Note, that I will live 'r_avg' as missing values because they are not scatter-plot points from the portfolios.

In [ ]:
(avg_exmkt,avg_rf) = port[['exmkt','rf']].mean()
est = pd.concat([est,pd.DataFrame({'r_avg':[np.nan,np.nan],
                               'beta':[0,1.8]})])

est['sml'] = avg_rf + est['beta']*avg_exmkt
est

**Step 4:** Plot using `pandas` and `matplotlib`. 

Note, I am going to try to make the output pretty using ggplot style, color changes, and even a little $\LaTeX$.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('default')

red = "#C80815"
blue = "#1560BD"

In [ ]:
ax = est.plot(y=['r_avg','sml'],x='beta',style=['o','-'],xlim=(0.0,1.8),
                  ylim=(0.0,1.8),figsize=(8,4.8),legend=False)

ax.lines[0].set_color(red)
ax.lines[1].set_color(blue)

**Plot Settings I Like**

In [ ]:
ax = est.plot(y=['r_avg','sml'],x='beta',style=['o','-'],xlim=(0.0,1.8),
                ylim=(0.0,1.8),figsize=(8,4.8),legend=False,markersize=5.5)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_ylabel(r'$\overline{r}_{p}$',fontsize=14)
ax.set_xlabel(r'$\hat{\beta}_p$',fontsize=14)

ax.set_yticks(np.arange(0,1.81,0.2))
ax.set_xticks(np.arange(0,1.81,0.2))

import matplotlib.ticker as mtick
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=1))

ax.lines[0].set_color(red)
ax.lines[1].set_color(blue)
ax.lines[1].set_linewidth(2)

# plt.savefig('sml.pdf',bbox_inches='tight')  